<a href="https://colab.research.google.com/github/jackychencw/MIE1624_Course_Project_Group19/blob/Lawrence-nlp-preprocess/MIE1624_project_train_keras2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/preproced_train_df.csv")

In [5]:
train_df = train_df.dropna(subset=['preproced_text'])

In [6]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(train_df['preproced_text'])
tokenized_text = tokenizer.texts_to_matrix(train_df['preproced_text'], mode='count')

In [7]:
X = np.concatenate([tokenized_text, train_df['polarities'].values.reshape(-1, 1), train_df['subjectivities'].values.reshape(-1, 1)], axis=1)
y = train_df['overall'].values

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import concatenate
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras import Input
from keras import Model
from keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.optimizers import Adam
class ReviewTextRegressor():
  def __init__(self, seq_size, meta_size, output_size, lr, embedding_size, epoch, batch_size):
    self.seq_size = seq_size
    self.meta_size = meta_size
    self.output_size = output_size
    self.epoch = epoch
    self.batch_size = batch_size
    embedding_vecor_length = 128
    self.model = Sequential()
    

    # 0.70
    nlp_input = Input(shape=(seq_size,)) 
    meta_input = Input(shape=(meta_size,))
    emb = Embedding(output_dim=embedding_vecor_length, input_dim=embedding_size, input_length=seq_size)(nlp_input) 
    nlp_out = Bidirectional(LSTM(128))(emb) 
    concat = concatenate([nlp_out, meta_input]) 
    regressor = Dense(32, activation='relu')(concat) 
    output = Dense(1, activation='relu')(regressor) 
    self.model = Model(inputs=[nlp_input , meta_input], outputs=[output])
    lr_schedule = ExponentialDecay(
            lr,
            decay_steps=1874,
            decay_rate=0.9,
            staircase=True)

    opt = Adam(learning_rate=lr_schedule)
    self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])
    self.early_stop = EarlyStopping(monitor='loss', patience=5)
    self.check_pt = ModelCheckpoint(filepath="/content/drive/MyDrive/Colab Notebooks/lr001.ckpt", save_weights_only=True,
                                             verbose=1)

    # # 0.83
    # nlp_input = Input(shape=(seq_size,)) 
    # meta_input = Input(shape=(meta_size,))
    # emb = Embedding(output_dim=embedding_vecor_length, input_dim=embedding_size, input_length=seq_size)(nlp_input) 
    # nlp_out = Bidirectional(LSTM(128))(emb) 
    # concat = concatenate([nlp_out, meta_input]) 
    # regressor = Dense(32, activation='relu')(concat) 
    # output = Dense(1, activation='relu')(regressor) 
    # self.model = Model(inputs=[nlp_input , meta_input], outputs=[output])
    # lr_schedule = ExponentialDecay(
    #         lr,
    #         decay_steps=10000,
    #         decay_rate=0.96,
    #         staircase=True)

    # opt = Adam(learning_rate=lr_schedule)
    # self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mse'])

    # self.model.add(Embedding(embedding_size, embedding_vecor_length, input_length=self.input_size))
    # # self.model.add(Dense(128, input_shape=(input_size,), activation='relu'))
    # self.model.add(Dropout(0.2))
    # self.model.add(LSTM(10))
    # self.model.add(Dropout(0.2))
    # self.model.add(Dense(1, activation='relu'))
    # opt = Adam(learning_rate=lr)
    # self.model.compile(loss="mean_squared_error", optimizer=opt, metrics=[])

  def fit(self, seq, meta, y):
    self.model.fit([seq, meta], y, epochs=self.epoch, batch_size=self.batch_size,
                   callbacks=[self.early_stop, self.check_pt])

  def predict(self, seq, meta):
    seq = seq.reshape(-1, self.self.seq_size)
    meta = seq.reshape(-1, self.self.meta_size)
    return self.model.predict([seq, meta])

In [45]:
num_word = 500

In [46]:
def train(X_train, y_train, num_word, lr=0.01, epoch=30, batch_size=64):
  X_txt_train = X_train[:, :num_word]
  # model = ReviewTextRegressor(num_word, 2, 1, lr=lr, embedding_size=int(X_txt_train.max()+1), epoch=epoch, batch_size=batch_size)
  model = ReviewTextRegressor(num_word, 2, 1, lr=lr, embedding_size=2000, epoch=epoch, batch_size=batch_size)
  model.fit(X_txt_train, X_train[:, num_word:], y_train)
  return model
  
  # sentiment_model = ReviewSentimentRegressor(input_size=X_train_all.shape[1], output_size=1, lr=lr, epoch=epoch, batch_size=batch_size)
  # sentiment_model.fit(X_train_all, y_train)

In [ ]:
model = train(X_train, y_train, num_word)

Epoch 1/30
1874/1874 [==============================] - 129s 67ms/step - loss: 1.2186 - mse: 1.2186

Epoch 00001: saving model to /content/drive/MyDrive/Colab Notebooks/lr001.ckpt
Epoch 2/30
1874/1874 [==============================] - 125s 67ms/step - loss: 0.8929 - mse: 0.8929

Epoch 00002: saving model to /content/drive/MyDrive/Colab Notebooks/lr001.ckpt
Epoch 3/30
  99/1874 [>.............................] - ETA: 1:53 - loss: 0.8685 - mse: 0.8685